<a href="https://colab.research.google.com/github/kaya-atakan/sentiment-analysis/blob/colab/25GB_E_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is based on the following notebook:

https://www.kaggle.com/shirellamosi/sentiment-analysis-nlp

Based on user review, I'm trying to predict whether the user would recommend the product to someone else. For that purpose, I'm implementing Random Forest Classifier.

In [3]:
import pandas as pd
from itertools import cycle
import numpy as np
import datetime as dt
from sklearn import linear_model
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None 
import seaborn as sns; sns.set()
import pickle
import plotly.express as px


from google.colab import drive
drive.mount('/content/drive', force_remount=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [5]:
%cd drive/My\ Drive/kaggle/E-commerce_Review

/content/drive/My Drive/kaggle/E-commerce_Review


In [9]:
data = pd.read_csv('E-commerce_Reviews.csv')

In [10]:
print(data.shape)
print(data.info())
data = data.rename(columns={"Review Text": "review", "Recommended IND": "Recommended"})

(23486, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB
None


In [11]:
data['Title'] = data['Title'].fillna("") 
data = data[~data['review'].isnull()] 

# data[data['review'].isnull()]
data.head(10)

,Unnamed: 0,Clothing ID,Age,Title,review,Rating,Recommended,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [12]:
#Positive Feedback Count: Positive Integer documenting the number of other customers who found this review positive.
data['review'] = data['Title'] + ' ' + data['review']

data['Review_length'] = data['review'].apply(len) #Make sure that there is no NAN because they are represented as float in python.
data = data.drop(labels=['Title'] , axis=1)

data.head(10)

,Unnamed: 0,Clothing ID,Age,review,Rating,Recommended,Positive Feedback Count,Division Name,Department Name,Class Name,Review_length
0,0,767,33,Absolutely wonderful - silky and sexy and com...,4,1,0,Initmates,Intimate,Intimates,54
1,1,1080,34,Love this dress! it's sooo pretty. i happen...,5,1,4,General,Dresses,Dresses,304
2,2,1077,60,Some major design flaws I had such high hopes ...,3,0,0,General,Dresses,Dresses,524
3,3,1049,50,"My favorite buy! I love, love, love this jumps...",5,1,0,General Petite,Bottoms,Pants,141
4,4,847,47,Flattering shirt This shirt is very flattering...,5,1,6,General,Tops,Blouses,209
5,5,1080,49,Not for the very petite I love tracy reese dre...,2,0,4,General,Dresses,Dresses,512
6,6,858,39,Cagrcoal shimmer fun I aded this in my basket ...,5,1,1,General Petite,Tops,Knits,517
7,7,858,39,"Shimmer, surprisingly goes with lots I ordered...",4,1,4,General Petite,Tops,Knits,519
8,8,1077,24,Flattering I love this dress. i usually get an...,5,1,0,General,Dresses,Dresses,177
9,9,1077,34,"Such a fun dress! I'm 5""5' and 125 lbs. i orde...",5,1,0,General,Dresses,Dresses,378


In [13]:
def to_sentiment(rating):
  rating = int(rating)
  if rating <= 2:
    return 0 #Negative
  elif rating == 3: # Neutral
    return 1
  else:
    return 2 # Positive

data['sentiment'] = data.Rating.apply(to_sentiment)

In [14]:
print(data['Recommended'].value_counts(normalize=True))

print(data['Rating'].value_counts(normalize=True))

1    0.818868
0    0.181132
Name: Recommended, dtype: float64
5    0.553862
4    0.216775
3    0.124685
2    0.068416
1    0.036262
Name: Rating, dtype: float64


In [15]:
from textblob import *
data['Polarity'] = data['review'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [16]:
import nltk
nltk.download('stopwords')
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def process_review(review):
    """Process review function.
    Input:
        review: a string containing a review
    Output:
        reviews_clean: a list of words containing the processed review

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    review = re.sub(r'\$\w*', '', review)
    # remove old style rereview text "RT"
    review = re.sub(r'^RT[\s]+', '', review)
    # remove hyperlinks
    review = re.sub(r'https?:\/\/.*[\r\n]*', '', review)
    # remove hashtags
    # only removing the hash # sign from the word
    review = re.sub(r'#', '', review)
    # remove numbers
    review = re.sub(r'[0-9]+', '', review)
    # tokenize reviews
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    review_tokens = tokenizer.tokenize(review)

    reviews_clean = []
    for word in review_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # reviews_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            reviews_clean.append(stem_word)

    return reviews_clean

In [18]:
data['new_review'] = data.review.apply(process_review)
data['new_review_string'] = [','.join(map(str, l)) for l in data['new_review']]


In [19]:
bow_transformer = CountVectorizer()
bow_transformer.fit(data['new_review_string'])
print(bow_transformer.vocabulary_)

X_bow = bow_transformer.transform(data['new_review_string'])

{'absolut': 29, 'wonder': 9987, 'silki': 7882, 'sexi': 7695, 'comfort': 1774, 'love': 5107, 'dress': 2628, 'sooo': 8168, 'pretti': 6758, 'happen': 3937, 'find': 3233, 'store': 8414, 'glad': 3691, 'bc': 697, 'never': 5807, 'would': 10037, 'order': 6099, 'onlin': 6049, 'petit': 6442, 'bought': 1039, 'length': 4907, 'hit': 4094, 'littl': 5016, 'knee': 4728, 'definit': 2285, 'true': 9193, 'midi': 5438, 'someon': 8152, 'truli': 9195, 'major': 5218, 'design': 2356, 'flaw': 3305, 'high': 4062, 'hope': 4144, 'realli': 7042, 'want': 9729, 'work': 10008, 'initi': 4410, 'small': 8060, 'usual': 9539, 'size': 7933, 'found': 3456, 'outrag': 6153, 'fact': 3075, 'could': 1984, 'zip': 10173, 'reorder': 7196, 'medium': 5369, 'ok': 6026, 'overal': 6168, 'top': 9077, 'half': 3903, 'fit': 3257, 'nice': 5828, 'bottom': 1033, 'tight': 9003, 'layer': 4843, 'sever': 7690, 'somewhat': 8160, 'cheap': 1487, 'net': 5802, 'imo': 4306, 'sewn': 7694, 'directli': 2448, 'zipper': 10177, 'favorit': 3151, 'buy': 1258, 'j

In [20]:
tfidf_vec = TfidfVectorizer()
tfidf_array = tfidf_vec.fit(data['new_review_string'])

tf-idf fit return np array, whereas fit_transform creates a sparse matrix.

In [21]:
X_tfidf = tfidf_vec.fit_transform(data['new_review_string'])
X_features = pd.DataFrame(X_tfidf.toarray())
X_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,10148,10149,10150,10151,10152,10153,10154,10155,10156,10157,10158,10159,10160,10161,10162,10163,10164,10165,10166,10167,10168,10169,10170,10171,10172,10173,10174,10175,10176,10177,10178,10179,10180,10181,10182,10183,10184,10185,10186,10187
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.392558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.131079,0.0,0.0,0.0,0.120825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.311981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df = data[['Rating','Recommended', 'Positive Feedback Count', 'Review_length', 'sentiment', 'Polarity']]

In [23]:
df_all = pd.merge(df, X_features, left_index=True, right_index=True)

In [24]:
df_all.head()

,Rating,Recommended,Positive Feedback Count,Review_length,sentiment,Polarity,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,...,10148,10149,10150,10151,10152,10153,10154,10155,10156,10157,10158,10159,10160,10161,10162,10163,10164,10165,10166,10167,10168,10169,10170,10171,10172,10173,10174,10175,10176,10177,10178,10179,10180,10181,10182,10183,10184,10185,10186,10187
0,4,1,0,54,2,0.633333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.392558,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,4,304,2,0.339583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0,524,1,0.073209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.131079,0.0,0.0,0.0,0.120825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,1,0,141,2,0.560714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,6,209,2,0.512891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
X = df_all.drop('Recommended', axis=1)
y = df_all.Recommended

In [26]:
from sklearn.model_selection import train_test_split as split
X_train, X_test, y_train, y_test = split(X,y, test_size=0.3, stratify=y, random_state=111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15273, 10193), (6546, 10193), (15273,), (6546,))

In [27]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)
 
 
y_pred = text_classifier.predict(X_test)
 


In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1091   94]
 [ 321 5040]]
              precision    recall  f1-score   support

           0       0.77      0.92      0.84      1185
           1       0.98      0.94      0.96      5361

    accuracy                           0.94      6546
   macro avg       0.88      0.93      0.90      6546
weighted avg       0.94      0.94      0.94      6546

0.9366025053467767


In [29]:
y_test.shape

(6546,)

In [30]:
df_confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,0,1,All
Actual,,,
0,1091,94,1185
1,321,5040,5361
All,1412,5134,6546


In [31]:
df_conf_norm = df_confusion / df_confusion.sum(axis=1)
df_conf_norm

Predicted,0,1,All
Actual,,,
0,0.460338,0.008767,0.090513
1,0.135443,0.470062,0.409487
All,0.595781,0.478829,0.500000
